In [1]:
pip install sec-api

In [14]:
from sec_api import QueryApi

queryApi = QueryApi(api_key="c50d2b3d65b4de5d80efb7f4f227737bf0ba463c7c477d4cf18fb5df59784168")

In [15]:
companies = ["AAPL", "MSFT", "GOOG"]
years = ["2021", "2020", "2019"]


In [16]:
for company in companies:
    for year in years:
        query = {
            "query": {
                "query_string": {
                    "query": f"ticker:{company} AND formType:\"10-K\" AND NOT formType:\"NT 10-K\" AND NOT formType:\"10-K/A\" AND filedAt:[{year}-01-01 TO {year}-12-31]",
                    "time_zone": "America/New_York"
                }
            },
            "from": "0",
            "size": "10",
            "sort": [{ "filedAt": { "order": "desc" } }]
        }



In [3]:
# query = {
#   "query": { "query_string": {
#       "query": "formType:\"10-K\" " +
#                "AND NOT formType:\"NT 10-K\" " +
#                "AND NOT formType:\"10-K/A\" " +
#                "AND filedAt:[2021-01-01 TO 2021-12-31]",
#       "time_zone": "America/New_York"
#   } },
#   "from": "0",
#   "size": "10",
#   "sort": [{ "filedAt": { "order": "desc" } }]
# }

# response = queryApi.get_filings(query)

In [17]:
response = queryApi.get_filings(query)


In [22]:
response

{'total': {'value': 1, 'relation': 'eq'},
 'query': {'from': 0, 'size': 10},
 'filings': [{'id': '1ed6cb98d658b4ab158f267ed44ddaf6',
   'accessionNo': '0001652044-19-000004',
   'cik': '1652044',
   'ticker': 'GOOG',
   'companyName': 'Alphabet Inc.',
   'companyNameLong': 'Alphabet Inc. (Filer)',
   'formType': '10-K',
   'description': 'Form 10-K - Annual report [Section 13 and 15(d), not S-K Item 405]',
   'filedAt': '2019-02-04T21:06:38-05:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1652044/0001652044-19-000004-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/goog10-kq42018.htm',
   'entities': [{'companyName': 'Alphabet Inc. (Filer)',
     'cik': '1652044',
     'irsNo': '611767919',
     'stateOfIncorporation': 'DE',
     'fiscalYearEnd': '1231',
     'type': '10-K',
     'ac

In [23]:
response['filings'][0]['linkToTxt']

'https://www.sec.gov/Archives/edgar/data/1652044/000165204419000004/0001652044-19-000004.txt'

Let's inspect the Query API response and print a subset of properties of each filing, namely `formType` and `periodOfReport`. The filings live in the `response["filings"]` list.

Feel free to skip the next lines if you're familiar with the use of `map` and `lambda`. The `map` function applies the `lambda` function to every filing in the list of filings. The `lambda` function simply returns a new dict for each filing by extracting the `formType` and `periodOfReport` property from the filing, and setting both values as the values of the new dict. Finally, the result of the `map` function is converted into a new list.

In [18]:
list(map(lambda x: {"formType": x["formType"], "periodOfReport": x["periodOfReport"]}, response["filings"]))

[{'formType': '10-K', 'periodOfReport': '2018-12-31'}]

In [19]:
period_by_cik = {}
period_by_ticker = {}

for filing in response["filings"]:
  cik, ticker, periodOfReport = filing["cik"], filing["ticker"], filing["periodOfReport"]

  if not cik in period_by_cik:
    period_by_cik[cik] = []

  if not periodOfReport in period_by_cik[cik]:
    period_by_cik[cik].append(periodOfReport)

  if len(ticker) > 0:
    if not ticker in period_by_ticker:
      period_by_ticker[ticker] = []

    if not periodOfReport in period_by_ticker[ticker]:
      period_by_ticker[ticker].append(periodOfReport)

In [20]:
period_by_cik

{'1652044': ['2018-12-31']}

In [21]:
period_by_ticker

{'GOOG': ['2018-12-31']}